# Homework 1: Numpy, Scipy, Pandas

### Due Monday Feb 5, 2018 @ 2pm


## #0: Get set up with your environment to work on and submit homework

a. Create a new homework repository at github

<img src="data/screenshot_1.png">

Name your repo something sensible (e.g., python-ay250-homeworks). Given your Berkeley affiliation you should be able to get private repos if you'd like.

<img src="data/screenshot_2.png">


b. Clone this repo locally and make a directory for this week's homework:

```bash
cd /class/directories ## this will be different on your machine...whereever you want to keep your work.

# change <username> to your github username:
git clone https://github.com/<username>/python-ay250-homework.git
cd python-ay250-homework
mkdir hw_1
echo "hw_1 README" >> hw_1/README.md
git add hw_1/README.md
git commit hw_1/README.md -m "added hw_1 directory"
git push
```

c. Copy this notebook into your `hw_1` folder from a local version of the python-seminar repo

```bash
cd /class/directories
git clone https://github.com/profjsb/python-seminar.git 
cd python-seminar
git pull
cp -r Homeworks/hw_1/* /class/directories/python-ay250-homework/hw_1/
```

d. Get working! Be sure to check in your work as often as you'd like

```bash
cd /class/directories/python-ay250-homework
git add hw_1/<whatever>
git commit <whatever> -m "this is a check in"
```

e. To submit your work, go to this form to register your homework Repo URL (https://goo.gl/forms/4DTPxonYBAxmfxJG2). If you've made your repo private, you'll need to add us as collaborators to the repo.  Our Github handles are `@profjsb` and `@chelseaharris`.

## #1:  Super-resolution imaging 

Obtaining several snapshots of the same scene, from microscopes to telescopes, is useful for the postprocessing increase of signal to noise: by summing up imaging data we can effectively beat down the noise. Interestingly, if we image the same scene from different vistas we can also improve the clarity of the combined image. Being able to discern features in a scene from this combination effort is sometimes called super-resolution imaging.

Here, we'll combine about 4 seconds of a shaky video to reveal the statement on a license plate that is not discernable in any one frame.

<img src="Data/im2-1.png">

A tarball of the data is at: https://drive.google.com/open?id=0B4vIeCR-xYNnbXFJTTVlVnpUZkk

```bash
tar -xvzf homework1_data.tgz  # do NOT check this files into git...
```

**Problem 1** Read in each image into a `numpy` array. Resize each frame to be 3 times larger in each axis (ie. 9 times larger images). Using `scipy.signal.fftconvolve` find the offsets of each frame with respect to the first frame. Report those offsets to 2 decimal places. 

   - Hint1: you'll need to figure out how to resize a numpy array
   - Hint2: you'll want to reverse the second image when doing the convolution: `scipy.signal.fftconvolve(im1, im2[::-1, ::-1])`
   - Hint3: you'll need to figure out how to identify the peak of the fft convolution to find the offsets between images

In [1]:
import matplotlib.pylab as plt
import PIL
from scipy import ndimage

# import the images
import os.path
imageDir = 'C:\\Users\\Liz\\Documents\\VSyear3\\AY250'
imageDir = os.path.join(imageDir, 'python-seminar', 'Homeworks', 'Ay250-homework', 'homework1_data', 'Data')

In [2]:
import numpy as np
import PIL
import matplotlib.pyplot as plt 
import scipy.signal as scisig
import scipy.misc


print('imports over')

directory = os.fsencode(imageDir)
imDim1 = 288
imDim2 = 288
imDim3 = 1
numImages = 100
im2 = np.empty([imDim1*3, imDim2*3, imDim3*3, numImages])
shiftIm2 = np.empty([imDim1*3, imDim2*3, imDim3*3, numImages])
imageCount = 0
offset = np.empty([numImages])

print('starting for loop')
# read images in 
for file in os.listdir(directory):
    imageFile = os.fsdecode(file)
 
    if imageFile[:2] !="._":
        imageToOpen = os.path.join(imageDir, imageFile)
        im = scipy.ndimage.imread(imageToOpen, flatten=True)#, flatten=True)
        
        im = np.resize(im, (imDim1*3, imDim2*3, imDim3*3))
        im2[:,:,:,imageCount] = im
        currentIm = im
        
        if imageCount == 0:
            originalIm = im
        elif imageCount > 0:
            offsetConvolv = scisig.fftconvolve(originalIm, currentIm[::-1, ::-1])
            currentOffset = np.unravel_index(np.argmax(offsetConvolv), offsetConvolv.shape)
            currentOffset.shape
            
            offset[imageCount] = currentOffset

            
            shift = scipy.ndimage.interpolation.shift(currentIm, currentOffset)
            print(shift.shape)
            shiftIm2[:,:,:,imageCount] = shift
            #print(offset)
        print(imageCount)
        imageCount +=1
        
        
            
print('done!')
print('offsets:')
print(offset)

imports over
starting for loop


C:\Users\Liz\Anaconda3\lib\site-packages\ipykernel\__main__.py:22: DeprecationWarning: The Windows bytes API has been deprecated, use Unicode filenames instead


0


AttributeError: 'tuple' object has no attribute 'shape'

**Problem 2** Shift each image to register the frames to the original (expanded in size) frame. You should, in general, be shifting by subpixel offsets. You might want to look at `scipy.ndimage.interpolation.shift`

**Problem 3** Combine all the registered images to form a super-resolution image. What does the license plate read?

In [ ]:
tryIm = np.sum(shiftIm2, axis=0)
scipy.misc.imshow(tryIm)

# #2: An elementary introduction to spectral audio compression

In this problem, we'll explore the very basics of audio compression in the spectral domain using numpy and scipy. We'll do a bit of visualization with matplotlib, but since that is covered later in the course, we'll provide those functions for you.

Audio compression is a large and complex topic, and the design of a format for compressed audio such as the popular [MP3](http://en.wikipedia.org/wiki/MP3) is too complex to cover in detail here. However, we will introduce the basic tools that most such compression formats use, namely:

1. Converting the input signal to the frequency domain by taking a Fast Fourier Transform (FFT).

2. Dropping information in the frequency domain, resulting in a smaller amount of data.

3. Reconstructing back the signal in the time domain from this smaller representation of the signal.

Steps 1 and 2 above are the 'encoding' part of signal compression, and step 3 is the 'decoding' part. For this reason, the tools that perform these steps are typically referred to as signal 'codecs', short for encoders/decoders.

Note that here we say 'signal': while MP3 is an audio format, the same ideas apply to the compression of digital images with formats such as JPEG and video.  Virtually all multimedia technologies we use today, from audio players to cell phones, digital cameras and YouTubeVideo, are based on sophisticated extensions and applications of these simple ideas.

Let's first load the plotting tools and importing some tools we'll need later:

In [ ]:
%pylab inline
import numpy as np
import matplotlib.pyplot as plt

# we'll need some path manipulations later on
import os

We define a simple utility function to listen to audio files right in the browser:

In [ ]:
def Audio(fname):
    """Provide a player widget for an audio file.
    
    Parameters
    ==========
    fname : string
      Filename to be played.
      
    Warning
    =======
    
    Browsers cache audio very aggressively. If you change an
    audio file on disk and are trying to listen to the  new version, you 
    may want to 
    """
    from IPython.display import HTML, display
    
    # Find out file extension and deduce MIME type for audio format
    ext = os.path.splitext(fname)[1].replace('.', '').lower()
    mimetype = 'audio/' + ('mpeg' if ext == 'mp3' else ext)
    
    tpl = """<p>{fname}:</p>
<audio controls>
    <source src="files/{fname}" type="{mimetype}">

Your browser does not support the Audio element; you can play 
<a href="files/{fname}">this file</a> manually.
</audio>
"""
    display(HTML(tpl.format(**locals())))

We also define a convenience wrapper around `plt.specgram`, [matplotlib's spectrogram function](http://matplotlib.org/api/mlab_api.html#matplotlib.mlab.specgram), with a colorbar and control over the color limits displayed. This will make it easier to compare across different signals with the same colors for all inputs.

In [ ]:
def specgram_cbar(x, title=None, clim=(0, 80) ):
    """Plot spectrogram with a colorbar and range normalization.
    
    Call matplotlib's specgram function, with a custom figure size, 
    automatic colobar, title and custom color limits to ease 
    comparison across multiple figures.
    
    Parameters
    ==========
    x : array
      One-dimensional array whose spectrogram should be plotted.
      
    title : string
      Optional title for the figure.
      
    clim : 2-tuple
      Range for the color limits plotted in the spectrogram.
    """
    f = plt.figure(figsize=(10,3))
    plt.specgram(x, Fs=2) #plt.specgram(x, Fs=2)
    plt.colorbar()
    plt.clim(*clim)
    if title is not None:
        plt.title(title)
    plt.show()

**Problem 1**: Use the `Audio` function above to listen to the signal we will be experimenting with, a simple voice recording stored in the file `Homeworks/hw1/data/voice.wav`.

Note: if your browser doesn't support audio, you may try a different browser.   We've tested current versions of Chrome and Firefox, and it works OK with both.

In [ ]:
# your code here
voicesDir = 'data\\voice.wav'
Audio(voicesDir)

**Problem 2**: Write a function to compress a 1-d signal by dropping a fraction of its spectrum. 

You can drop the smallest components by setting their values to zero.

*Hints*: 

- look at the `np.fft` module, keeping in mind that your input signal is real.
- look at the `argsort` method of numpy arrays.

In [ ]:
def compress_signal(x, fraction):
    """Compress an input signal by dropping a fraction of its spectrum.
    
    Parameters
    ==========
    x : array
      1-d real array to be compressed
      
    fraction : float
      A number in the [0,1] range indicating which fraction of the spectrum
      of x should be zeroed out (1 means zero out the entire signal).
      
    Returns
    =======
    x_approx : array
      1-d real array reconstructed after having compressed the input.
    """
    # your code here
    transform = np.fft.fft(x)
    x_order = np.argsort(x)
    toTruncate = round(len(x)*fraction)
    for i in range(toTruncate):
        remove = x_order[-i]
        transform[remove] = 0
        
    x_approx = np.fft.ifft(transform)

    return x_approx

As a quick visual check (not that this is *not* a formal test of correctness), experiment with a simple random signal by changing the compression ratio and plotting both the signal and the compressed version:

In [ ]:
x = np.random.rand(128)


In [ ]:
fraction = 0.6  # play changing this in the 0-1 range

xa = compress_signal(x, fraction)

plt.figure(figsize=(12,3))
plt.plot(x, alpha=0.5, lw=2, label='original')
plt.plot(xa, lw=2, label='compressed {0:.0%}'.format(fraction))
plt.legend();

**Problem 3**: Write a function that will compress an audio file by a dropping a fraction of its spectrum, writing the output to a new file.

If the input file is named `a.wav` and the compression fraction is 0.9, the output file should be named `a_comp_0.9.wav`.

*Hints:* 

- look at the `scipy.io` module for routines dealing with files in `wav` format.

- you may need to use the `astype` method of numpy arrays to get the correct data type for `wav` files.

In [ ]:
def compress_wav(fname, fraction):
    """Compress an audio signal stored in an input wav file.
    
    The compressed signal is returned as a numpy array and automatically written 
    to disk to a new wav file.
    
    Parameters
    ==========
    fname : string
      Name of the input wav file
      
    fraction : float
      Fraction of input data to keep.
      
    Returns
    =======
    rate : int
      Bit rate of the input signal.

    x : array
      Raw data of the original input signal.
      
    x_approx : array
      Raw data of the compressed signal.
      
    new_fname : string
      Auto-generated filename of the compressed signal.
    """
    # import
    import scipy.io.wavfile as wav
    rate, x = wav.read(fname)
    x_approx = compress_signal(x, fraction)
    x_approx = x_approx.astype(x.dtype)

    removeDir = fname.partition("\\")[-1]
    new_fname = removeDir[:-4] + '_comp_' + str(fraction) + fname[-4:]
    
    # save compressed file 
    wav.write(new_fname, rate, x_approx)
    
    return rate, x, x_approx, new_fname

**Problem 4**: Study the effect of compressing the input file at different ratios: 0.1, 0.5, 0.75, 0.9, 0.95, 0.99.

Using the `OrderedDict` class from the [Python collections module](http://docs.python.org/2/library/collections.html#collections.OrderedDict), store the uncompressed signal as well as the compressed array and filename for each compression ratio.

You will create an `OrderedDict` called `voices`, with:

- keys: compression ratios
- values: pairs of (x, filename) where x is the compressed audio and filename is the name of the compressed file.

In [ ]:
import collections

voices = collections.OrderedDict()
fractions = [0.1, 0.5, 0.75, 0.9, 0.95, 0.99]

voicesDir = 'data\\voice.wav'

for fract in fractions:
    rate, x, x_approx, new_fname = compress_wav(voicesDir, fract)
    voices[fract] = (x_approx, new_fname)
    


**Problem 5**: Loop over the `voices` dict, and for each one generate an audio player as well as a spectrogram.  Observe how the spectrogram changes, and listen to each file.  At what ratio do you stop understanding the recording?

In [ ]:
import scipy.io.wavfile as wav

for comp in voices:
    fname = voices[comp][1]
    print(fname)
    
    rate, x = wav.read(fname)
    
    specgram_cbar(x, title=None)
    Audio(fname)


##### Summary
After removing ~75% of the audio file, it becomes unable to interpret

# #3: Armchair Astronomer

Often times, people act as good sensors of the physical universe. We can use Google Trends data to help us determine some fundamental parameters of the Solar System.

**Problem 1**: Using just the CSV file we created in the pandas lecture (`merged_data.csv`) and some frequency analysis tools in `scipy` to determine:

   - the number of days in a year
   - the period of the moon's orbit around the Earth
   
Hint: `from scipy.signal.spectral import lombscargle` (you'll want to normalize your signal so it has a mean of zero.)

In [ ]:
import pandas as pd
from datetime import datetime

fred = pd.read_csv('multiTimeline.csv', index_col=0, parse_dates=[0])

moon = pd.read_csv('full_moon.csv', index_col=0, parse_dates=[0])

df = fred.merge(moon, left_index=True, right_index=True, how = 'right')
df.to_csv("merged_data.csv")

# print(df.index[0])
# print(pd.to_numeric(df.index[0]))
# dates = [datetime.strptime(str(x)[:10], '%Y-%m-%d') for x in df.index]
df

In [ ]:
# your code here
from scipy.signal import lombscargle
# Lomb-scargle periodogram was invented to find and test the significance of weak
# periodic signals and uneven temporal sampling 

a = lombscargle(dates, df["full moon"], 1000, normalize=True)

# #4: Reproducing some insights about the Election

Nate ("not a genius, just a Bayesian") Silver writes often about polls and their utility of predicting elections. One of the things he emphasized during the 2016 campaign is that even "large" polls of people with a consistent lead for one candidate will show wild swings in any given window in time.

**Problem 1**: Using Pandas and `numpy`, try to reproduce this plot from a Nate Silver Tweet qualitatively using the same assumptions.

<img src="https://www.evernote.com/l/AUU-KkH3Z-lBdKokzCzVIUe53YZLQ9e_AuMB/image.png">

https://twitter.com/NateSilver538/status/769565612955824128

#### Assumptions:
- steady mean of Clinton + 6
- 1000 people being polled over time

In [ ]:
# Assumptions:
# your code here

months = ['June', 'July', 'Aug', 'Sept', 'Oct', 'Nov']
daysInMonth = 30 # approx
numPolls = (daysInMonths)*6 # 6 months
percentClinton = np.empty([numPolls])

#binomial (200, 0.56)
# numpy.random.binomial
tryBi = np.random.binomial(200, 0.53) # sample 200 voters with a 53% of choosing Clinton (47% trump -> 6pt lead?)

poll = 0
for month in months:
    for days in range(daysInMonth):
        pollOfDays = np.random.binomial(200, 0.53)
        pollOfDays = ndimage.filters.convolve(pollOfDays, 0.2)
        
        percentClinton[poll] = pollOfDays
        poll += 1


**Problem 2**: Clearly, even with a 6% point lead, there's a chance that this sort of poll would show the other person in the lead. How much would ahead (in percent) would a candidate need to be to have a tracking poll never show the other candidate to be ahead over the course of a year (in your simulation)?

In [ ]:
# your code here

**Problem 3**: With a 3 and 6% lead, how many people would need to be polled in 1 day to have the rolling 5-day poll result always show the leader ahead (over a year)?

In [ ]:
# your code here